In [0]:
pip install azure-storage-blob

Python interpreter will be restarted.
Requirement already satisfied: azure-storage-blob in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1682ce54-ed7f-41d8-acf2-58b1df28c332/lib/python3.8/site-packages (12.8.1)
Requirement already satisfied: cryptography>=2.1.4 in /databricks/python3/lib/python3.8/site-packages (from azure-storage-blob) (3.1.1)
Requirement already satisfied: msrest>=0.6.18 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1682ce54-ed7f-41d8-acf2-58b1df28c332/lib/python3.8/site-packages (from azure-storage-blob) (0.6.21)
Requirement already satisfied: azure-core<2.0.0,>=1.10.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1682ce54-ed7f-41d8-acf2-58b1df28c332/lib/python3.8/site-packages (from azure-storage-blob) (1.17.0)
Requirement already satisfied: six>=1.4.1 in /databricks/python3/lib/python3.8/site-packages (from cryptography>=2.1.4->azure-storage-blob) (1.15.0)
Requirement already satisfied: cffi!=1.11.3,>=1.8 in /databricks/python3/lib/python3.8/site-packages (from cryptography>=2.1.4->azure-storage-blob) (1.14.3)
Requirement already satisfied: isodate>=0.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1682ce54-ed7f-41d8-acf2-58b1df28c332/lib/python3.8/site-packages (from msrest>=0.6.18->azure-storage-blob) (0.6.0)
Requirement already satisfied: requests~=2.16 in /databricks/python3/lib/python3.8/site-packages (from msrest>=0.6.18->azure-storage-blob) (2.24.0)
Requirement already satisfied: requests-oauthlib>=0.5.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1682ce54-ed7f-41d8-acf2-58b1df28c332/lib/python3.8/site-packages (from msrest>=0.6.18->azure-storage-blob) (1.3.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from msrest>=0.6.18->azure-storage-blob) (2021.5.30)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi!=1.11.3,>=1.8->cryptography>=2.1.4->azure-storage-blob) (2.20)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests~=2.16->msrest>=0.6.18->azure-storage-blob) (2.10)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests~=2.16->msrest>=0.6.18->azure-storage-blob) (3.0.4)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests~=2.16->msrest>=0.6.18->azure-storage-blob) (1.25.11)
Requirement already satisfied: oauthlib>=3.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1682ce54-ed7f-41d8-acf2-58b1df28c332/lib/python3.8/site-packages (from requests-oauthlib>=0.5.0->msrest>=0.6.18->azure-storage-blob) (3.1.1)
WARNING: You are using pip version 20.2.4; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-1682ce54-ed7f-41d8-acf2-58b1df28c332/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient
from pyspark.sql.types import StructField, StructType, StringType, FloatType, IntegerType, ShortType
import pandas as pd
import os

In [0]:
# start spark session
spark = SparkSession.builder.appName("app").getOrCreate()
spark.conf.set("fs.azure.account.key.cahospitalstorage.blob.core.windows.net", \
    "w3EYeDH//L+J0uscsn3BYDatbFCcB7CUXK4MfOSNmwc16v3WAQ7EPcw2IpchUrelLVuowjfugmv99e0Os8J5Sw==")

In [0]:
# connect to blob storage and download csv file
container = "hospitals"
blob_location = "raw_data/facilityprofile_2021-08-10.csv"

blob_service_client = BlobServiceClient.from_connection_string("DefaultEndpointsProtocol=https;AccountName=cahospitalstorage;AccountKey=w3EYeDH//L+J0uscsn3BYDatbFCcB7CUXK4MfOSNmwc16v3WAQ7EPcw2IpchUrelLVuowjfugmv99e0Os8J5Sw==;EndpointSuffix=core.windows.net")
container_client = blob_service_client.get_container_client(container)
blob_data = container_client.get_blob_client(blob_location)
with open("temp.txt", "wb") as temp:
  data = blob_data.download_blob()
  data.readinto(temp)

In [1]:
# read csv file
schema = StructType([
    StructField('facility_desc', StringType(), True),
    StructField('site_address1', StringType(), True),
    StructField('site_address2', StringType(), True),
    StructField('site_city', StringType(), True),
    StructField('site_zip', StringType(), True),
    StructField('site_x_coordinate', FloatType(), True),
    StructField('site_y_coordinate', FloatType(), True),
    StructField('oshpd_id', StringType(), True),
    StructField('license_type_desc', StringType(), True),
    StructField('license_category_desc', StringType(), True),
    StructField('license_number', StringType(), True)
])

pdf = pd.read_csv("temp.txt")
pdf = pdf.iloc[:,:11]
df = spark.createDataFrame(pdf, schema=schema)

NameError: name 'StructType' is not defined

In [0]:
# Export df_final to blob storage
output = "clean_data_hospitals"
df.write.option("header", "true").mode("overwrite").parquet(output)
for file in os.listdir("/dbfs/" + output):
  try:
    with open("/dbfs/%s/%s" % (output, file), "rb") as data:
        container_client.upload_blob(name="%s/%s" % (output, file), data=data, overwrite=True)
  except Exception as e:
    print(e)

In [0]:
dbutils.notebook.exit("%s/%s" % (container, output))

hospitals/clean_data_hospitals